# Introduction to Algorithms, 4th Edition: Part III Data Structures - Chapter 10 Elementary Data Structures - Section 11.2 Hash Tables

The downside of direct addressing is apparent: if the universe $U$ is large or infinite, storing a table $T$ of size $|U|$ may be impractical, or even impossible, given the memory available on a typical computer. Furthermore, the set $K$ of keys *actually stored* may be so small relative to $U$ that most of the space allocated for $T$ would be wasted.

When the set $K$ of keys stored in a dictionary is much smaller than the universe $U$ of all possible keys, a hash table requires much less storage than a direct-address table. Specifically, the storage requirement reduces to $\Theta(|K|)$ while maintaining the benefit that searching for an element in the hash table still requires only $O(1)$ time. The catch is that this bound is for the *average-case time*, whereas for direct addressing it holds for the *worst-case time*.

With direct addressing, an element with key $k$ is stored in slot $k$, but with hashing, we use a **hash function** $h$ to compute the slot number from the key $k$, so that the element goes into slot $h(k)$. The hash function $h$ maps the universe $U$ of keys into the slots of a **hash table** $T[0 : m - 1]$:
$$
h : U \to \{0, 1, \ldots, m - 1\},
$$
where the size $m$ of the hash table is typically much less than $|U|$. We say that an element with key $k$ **hashes** to slot $h(k)$, and we also say that $h(k)$ is the **hash value** of key $k$. Figure 11.2 illustrates the basic idea. The hash function reduces the range of array indices and hence the size of the array. Instead of a size of $|U|$, the array can have size $m$. An example of a simple, but not particularly good, hash function is $h(k) = k\bmod{m}$

<img src="Hash Table Idea.png" alt="Idea of a Hash Table" width="750"/>

There is one hitch, namely that two keys may hash into the same slot. We call this situation a **collision**. Fortunately, there are effective techniques for resolving the conflict created by collisions.

Of course, the ideal solution is to avoid collisions altogether. We might try to achieve this goal by choosing a suitable hash function $h$. One idea is to make $h$ appear to be "random," thus avoiding collisions or at least minimizing their number. The very term "to hash," evoking images of random mixing and chopping, captures the spirit of this approach. (Of course, a hash function $h$ must be deterministic in that a given input $k$ must always produce the same output $h(k)$.) Because $|U| > m$, however, there must be at least two keys that have the same hash value and avoiding collisions altogether is impossible. Thus, although a well-designed, "random"-looking hash function can reduce the number of collisions, we still need a method for resolving the collisions that do occur.

The remainder of this section first presents a definition of "independent uniform hashing," which captures the simplest notion of what it means for a hash function to be "random." It then presents and analyzes the simplest collision resolution technique, called chaining. Section 11.4 introduces an alternative method for resolving collisions, called open addressing.

## Independent Uniform Hashing
An "idea" hashing function $h$ would have, for each possible input $k$ in the domain $U$, an output $h(k)$ that is an element randomly and independently chosen uniformly from the range $\{0, 1, \ldots, m - 1\}$. Once a value $h(k)$ is randomly chosen, each subsequent call to $h$ with the same input $k$ yields the same output $h(k)$. 

We call such an ideal hash function an **independent uniform hash function**. Such a function is also called a **random oracle**. When hash tables are implemented with an independent uniform hash function, we say we are using **independent uniform hashing**.

Independent uniform hashing is an ideal theoretical abstraction, but it is not something that can reasonably be implemented in practice. Nonetheless, we'll analyze the efficiency of hashing under the assumption of independent uniform hashing and then present ways of achieving useful practical approximations to this ideal.

## Collision Resolution by Chaining
At a high level, you can think of hashing with chaining as a nonrecursive form of divide-and-conquer: the input set of $n$ elements is divided randomly into $m$ subsets, each of approximate size $\frac{n}{m}$. A hash function determines which subset an element belongs to. Each subset is managed independently as a list.

Figure 11.3 shows the idea behind **chaining**: each nonempty slot points to a linked list, and all the elements that hash to the same slot go into that slot's linked list. Slot $j$ contains a pointer to the head of the list of all stored elements with hash value $j$. If there are no such elements, then slot $j$ contains $\texttt{nil}$.

<img src="Chaining.png" alt="Hash Table Chaining" width="750"/>

When collisions are resolved by chaining, the dictionary operations are straightforward to implement. They appear below and use the linked-list procedures from Section 10.2. The worst-case running time for insertion is $O(1)$. The insertion procedure is fast in part because it assumes that the element $x$ being inserted is not already present in the table. To enforce this assumption, you can search (at additional cost) for an element whose key is $x.\textit{key}$ before inserting. For searching, the worst-case running time is proportional to the length of the list. (We'll analyze this operation more closely below.) Deletion takes $O(1)$ time if the lists are doubly linked, as in Figure 11.3. (Since $\texttt{Chained-Hash-Delete}$ takes as input an element $x$ and not its key $k$, no search is needed. If the hash table supports deletion, then its linked lists should be doubly linked in order to delete an item quickly. If the lists were only singly linked, deletion could take time proportional to the length of the list. With singly linked lists, both deletion and searching would have the same asymptotic running time.)

```
Chained-Hash-Insert(T, x)
[1] List-Prepend(T[h(x.key)], x)

Chained-Hash-Search(T, k)
[1] return List-Search(T[h(k)], k)

Chained-Hash-Delete(T, x)
[1] List-Delete(T[h(x.key)], x)
```

## Analysis of Hashing with Chaining
How well does hashing with chaining perform? In particular, how long does it take to search for an element with a given key?

Given a hash table $T$ with $m$ slots that stores $n$ elements, we define the **load factor** $\alpha$ for $T$ as $\frac{n}{m}$, that is, the average number of elements stored in a chain. Our analysis will be in terms of $\alpha$, which can be less than, equal to, or greater than $1$.

The worst-case behavior of hashing with chaining is terrible: all $n$ keys hash to the same slot, creating a list of length $n$. Then worsts-case time for searching is thus $\Theta(n)$ plus the time to compute the hash function - no better than using one linked list for all the elements. We clearly don't use hash tables for their worst-case performance.

The average-case performance of hashing depends on how well the hash function $h$ distributes the set of keys to be stored among the $m$ slots, on the average (meaning with respect to the distribution of keys to be hashed and with respect to the choice of hash function, if this choice is randomized). Section 11.3 discusses these issues, but for now we assume that any given element is equally likely to hash into any of the $m$ slots. That is, the hash function is **uniform**. We further assume that where a given element hashes to is *independent* of where any other elements hash to. In other words, we assume that we are using **independent uniform hashing**.

Because hashes of distinct keys are assumed to be independent, independent uniform hashing is **universal**: the chance that any two distinct keys $k_1$ and $k_2$ collide is at most $\frac{1}{m}$. Universality is important in our analysis and also in the specification of universal families of hash functions, which we'll see in Section 11.3.2.

For $j = 0, 1, \ldots, m - 1$, denote the length of the list $T[j]$ by $n_j$, so that
$$
n = n_0 + n_1 + \cdots + n_{m - 1}, \qquad \textbf{(11.1)}
$$
and the expected value of $n_j$ is $E[n_j] = \alpha = \frac{n}{m}$

We assume that $O(1)$ time suffices to compute the hash value $h(k)$, so that the time required to search for an element with key $k$ depends linearly on the length $n_{h(k)}$ of the list $T[h(k)]$. Setting aside the $O(1)$ time required to compute the hash function and to access slot $h(k)$

Note: Incomplete Section